In [27]:
import pandas as pd
from matplotlib.pyplot import show, figure, plot, subplots
import matplotlib.pyplot as plt
%matplotlib osx
import seaborn as sns
import numpy as np
from scipy import stats

In [2]:
data = pd.read_csv("data/sp500.csv")
data = data.set_index(pd.DatetimeIndex(data["Date"]))
data.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2014-07-17', '2014-07-18', '2014-07-21', '2014-07-22',
               '2014-07-23', '2014-07-24', '2014-07-25', '2014-07-28',
               '2014-07-29', '2014-07-30'],
              dtype='datetime64[ns]', name='Date', length=1151, freq=None)

In [112]:
data["Volume"].describe()


count    1.151000e+03
mean     3.839843e+09
std      9.278361e+08
min      1.025000e+09
25%      3.298250e+09
50%      3.722320e+09
75%      4.265890e+09
max      1.061781e+10
Name: Volume, dtype: float64

In [121]:

data["Year"] = data.index.to_period("A")
data["Month"] = data.index.to_period("M")
data["Day"] = data.index.weekday
data["Q"] = data.index.to_period("Q").strftime('Q%q')
data["PreviousVolume"] = data["Volume"].shift(1)


In [58]:
data["Volume"].plot()


In [99]:

params = stats.lognorm.fit(volume.values)
print(params)

sns.distplot(data.Volume.iloc[400:430], hist=True, kde=False, fit=stats.lognorm, bins=100)


(0.14110334442523884, -2550422425.4460354, 6325247618.913096)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [78]:
volume = data.Volume
previous_end = 0

periods_num = 2
items_num = len(volume.index)
period_length = int(items_num/periods_num)

xlim = (volume.min(), volume.max())

figure()
for period in range(periods_num):
    date_range = pd.date_range(previous_end, periods=period_length)
    start = date_range.min()
    # previous_end = date_range.max()
    start = previous_end
    previous_end = start + period_length
    label = "%s - %s" % (start, previous_end)
    volume.reindex(date_range).plot.kde(label=label, legend=True, xlim=xlim)
    # sns.distplot(volume.iloc[start:previous_end], fit=stats.lognorm, kde=False, hist=False)

In [124]:
data[["Day", "Volume"]].boxplot(by="Day")

In [119]:
data["t-1"] = data["Volume"].shift(1)
sns.lmplot(x="Volume", y="t-1", data=data, order=1)
plt.grid(True)
plt.ylim(data["Volume"].min(), data["Volume"].max())
plt.xlim(data["Volume"].min(), data["Volume"].max())


(1025000000, 10617810000)

In [16]:
q = np.percentile(data["Volume"], [20, 80])
q_low, q_high = q[0], q[1]

data["below"] = (data["Volume"] <= q_low).astype(float)
data["above"] = (data["Volume"] >= q_high).astype(float)

data["below_t-1"] = data["below"].shift(1)
data["above_t-1"] = data["above"].shift(1)
sns.regplot(x="below_t-1", y="Volume", data=data)
sns.regplot(x="above_t-1", y="Volume", data=data)
